In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from sklearn import preprocessing
le = preprocessing.LabelEncoder()


In [2]:
PATH = './DataSet_w_NA.xlsx'
df = pd.read_excel(PATH, sheet_name="Испорченные факты")
df.dropna(inplace = True)
dict_of_classes) -> {0: 'A', 1: 'B', 2: 'C'}

In [3]:
df = pd.pivot_table(df, index='Факты.Товар ID', values = ['Продажи, шт', 'Продажи, руб', 'Маржинальная прибыль',
                                                             'Повторение заказа', 'Повторение товара'], 
                       aggfunc={'Продажи, шт': [np.median, np.sum],
                              'Продажи, руб': [np.median, np.sum],
                              'Повторение заказа': np.sum,
                              'Маржинальная прибыль': np.sum})

newname=df.columns.map('_'.join)
df.columns=newname
df=df.reset_index()

total_sale=df['Продажи, руб_sum'].sum()
df['Доля']=df['Продажи, руб_sum']/total_sale * 100
df = df.sort_values(by=('Продажи, руб_sum'), ascending=False)
df = df.assign(sum_d=df['Доля'].cumsum())

df.loc[(df['sum_d'] <= 80), 'ABC'] = 'A'
df.loc[(df['sum_d'] > 80) & (df['sum_d'] <= 95), 'ABC'] = 'B'
df.loc[(df['sum_d'] > 95), 'ABC'] = 'C'
le.fit(["A", "B", "C"])
df['class'] = le.transform(df['ABC'])

In [4]:
def Classifier_acc(df, features, classifier, scaler):
    X = df[features]
    Y = df['class']
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state = 1)
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    classifier.fit(X_train_scaled, y_train)
    y_pred = classifier.predict(X_test_scaled)
    
    print(f'Score is {classifier.score(X_train_scaled, y_train)}')
    print(f"Accuracy is {accuracy_score(y_test, y_pred)*100} %")
def Class_predict(params, classifier, scaler):
    data_scaled = scaler.transform(params)
    class_ = classifier.predict(data_scaled)
    return dict_of_classes[class_[0]]
    

In [5]:
scaler = StandardScaler()
data = [[25000, 10, 0.05]]


rfc = RandomForestClassifier()
Classifier_acc(df, ['Продажи, руб_median', 'Продажи, шт_sum', 'Доля'], rfc, scaler)
print(f'Предсказанный класс rfc: {Class_predict(data, rfc, scaler)}')

Score is 1.0
Accuracy is 99.73190348525469 %


C:\Users\Dmitry\AppData\Roaming\Python\Python37\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


NameError: name 'dict_of_classes' is not defined

In [ ]:
df[(df["ABC"] == "B")]